# Recetario de Snippets SQL

**Objetivo**: código SQL reutilizable para patrones comunes.

---

## 1. Agregaciones básicas

In [ ]:
-- Total unidades, ingresos, margen
SELECT SUM(cantidad) AS unidades_totales,
       SUM(cantidad * precio_unitario * (1 - descuento_pct/100.0)) AS ingresos_totales,
       SUM(cantidad * (precio_unitario - costo_unitario)) AS margen_total
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id;

## 2. Top N con RANK

In [ ]:
-- Top 5 productos por unidades
SELECT TOP 5 producto_id, nombre, unidades
FROM (
  SELECT p.producto_id, p.nombre, SUM(fv.cantidad) AS unidades
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY p.producto_id, p.nombre
) sub
ORDER BY unidades DESC;

## 3. Detección de nulos

In [ ]:
-- Clientes sin email
SELECT cliente_id, nombre
FROM dbo.dim_clientes
WHERE email IS NULL;

## 4. Ticket promedio

In [ ]:
-- Ticket promedio global
SELECT CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_promedio
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id;

## 5. LAG para variación día a día

In [ ]:
-- Unidades diarias y delta vs día anterior
WITH diario AS (
  SELECT fecha, SUM(cantidad) AS unidades
  FROM dbo.fact_ventas
  GROUP BY fecha
)
SELECT fecha, unidades,
       unidades - LAG(unidades) OVER (ORDER BY fecha) AS delta_unidades
FROM diario;

## 6. CTE para margen por producto

In [ ]:
WITH margen_prod AS (
  SELECT p.producto_id, p.nombre, SUM(fv.cantidad*(p.precio_unitario-p.costo_unitario)) AS margen
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY p.producto_id, p.nombre
)
SELECT * FROM margen_prod WHERE margen > 500;

## 7. Detección de productos sin ventas

In [ ]:
SELECT p.producto_id, p.nombre
FROM dbo.dim_productos p
LEFT JOIN dbo.fact_ventas fv ON p.producto_id = fv.producto_id
WHERE fv.producto_id IS NULL;

## 8. UNION de ingresos (ventas + suscripciones)

In [ ]:
SELECT fecha, 'VENTA' AS tipo, SUM(cantidad*p.precio_unitario) AS ingresos
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY fecha
UNION ALL
SELECT fecha_inicio AS fecha, 'SUSCRIPCION' AS tipo, SUM(mrr) AS ingresos
FROM dbo.fact_suscripciones
GROUP BY fecha_inicio;

## 9. Validación de anomalías

In [ ]:
-- Ventas con descuento > 50% o cantidad <= 0
SELECT venta_id, fecha, descuento_pct, cantidad
FROM dbo.fact_ventas
WHERE descuento_pct > 50 OR cantidad <= 0;

## 10. NULLIF para evitar división por cero

In [ ]:
-- Porcentaje de crecimiento robusto
SELECT fecha, unidades,
       CASE WHEN LAG(unidades) OVER (ORDER BY fecha) = 0 THEN NULL
            ELSE (unidades - LAG(unidades) OVER (ORDER BY fecha)) / NULLIF(LAG(unidades) OVER (ORDER BY fecha),0) END AS pct_crecimiento
FROM (
  SELECT fecha, SUM(cantidad) AS unidades FROM dbo.fact_ventas GROUP BY fecha
) sub;

---

*Snippets listos para copiar/adaptar en proyectos del curso.*

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---

## Navegación

[⬅️ Volver al Índice del Curso](../README.md)